## Data Loading

In [2]:
import h5py
import numpy as np
import pickle

data = h5py.File('/home/sunji/ANN/glove_200_angular/glove-200-angular.hdf5', 'r')
data_train = np.array(data['train'])
data_test = np.array(data['test'])
with open('/home/sunji/ANN/glove_200_angular/clusters_glove_200_angular.pkl', 'rb') as f:
    clusters = pickle.load(f)
with open('/home/sunji/ANN/glove_200_angular/ground_truth_glove_200_angular_0_0_0_5.pkl', 'rb') as f:
    ground_truth_total = pickle.load(f)

In [3]:
for c in clusters:
    print (len(c))

6850
8
21739
3030
40530
38454
28549
6
1
1
61036
2
61
37078
22370
12774
6432
4
1
15985
1
10685
1
6502
1
5
2
20906
1
19205
12962
8422
28661
36376
1
10057
4
9999
1
15894
54729
1
44225
6137
10548
8
5
47341
3
74975
11693
4255
2
13678
12827
2
9759
3593
49167
40
43652
2
45062
8884
18221
2
29121
1
8
1
1
13
22864
68070
12947
3
1
18729
15635
1
2
6
7
6310
10958
10
2
20297
2
2
2
1
2
29708
1
1
11361
3
4035
1


In [4]:
cluster_size = len(clusters)

In [5]:
ground_truth_total_level = [[[] for _ in range(10000)] for _ in range(cluster_size)]
for clus in range(cluster_size):
    for t in ground_truth_total[clus]:
        ground_truth_total_level[t[0]][t[1]].append(t)

In [21]:
angular_dist(data_train[1], data_train[8])

0.4716227830663988

In [6]:
centroids = []
for cluster in clusters:
    centroids.append(np.mean(cluster, 0))

In [7]:
centroids[0]

array([0.69149425, 0.93287356, 0.10942529, ..., 0.        , 0.        ,
       0.        ])

## Prepare Inputs

In [6]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy import spatial

from multiprocessing import Pool

def jaccard(x1, x2=None, eps=1e-8):
    x1 = x1.astype(bool)
    x2 = x2.astype(bool)
    return 1.0 - np.double(np.bitwise_and(x1, x2).sum()) / np.double(np.bitwise_or(x1, x2).sum())

def angular_dist(x1, x2=None, eps=1e-8):
    cosine_sim = 1 - spatial.distance.cosine(x1, x2)
    distance = np.arccos(cosine_sim) / 3.14159267
    return distance

def euclidean_dist_normalized(x1, x2=None, eps=1e-8):
    if x2 is None:
        return 1.0
    return np.sqrt(((x1 - x2) ** 2).mean())

def prepare_for_cluster(cluster_id):
    slot = 0.002
    batch_size = 128
    min_card = 1e10
    max_card = 0
    train_queries = []
    train_distances = []
    train_thresholds = []
    train_targets = []
    for query_id in range(8000):
        cardinality = 0
        for threshold_id, threshold in enumerate(np.arange(0.4, 0.5, slot)):
            cardinality += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality > 0:
                train_queries.append(data_test[query_id])
                train_distances.append([angular_dist(data_test[query_id], centroids[cluster_id])])
                train_thresholds.append([threshold+slot])
                train_targets.append([cardinality])
                card = np.log(cardinality)
                if card < min_card:
                    min_card = card
                elif card > max_card:
                    max_card = card

    test_queries = []
    test_distances = []
    test_thresholds = []
    test_targets = []
    for query_id in range(8000,10000):
        cardinality = 0
        for threshold_id, threshold in enumerate(np.arange(0.4, 0.5, slot)):
            cardinality += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality > 0:
                test_queries.append(data_test[query_id])
                test_distances.append([angular_dist(data_test[query_id], centroids[cluster_id])])
                test_thresholds.append([threshold+slot])
                test_targets.append([cardinality])
                card = np.log(cardinality)
                if card < min_card:
                    min_card = card
                elif card > max_card:
                    max_card = card
    
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(train_queries), torch.FloatTensor(train_distances), torch.FloatTensor(train_thresholds), torch.FloatTensor(train_targets)), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(test_queries), torch.FloatTensor(test_distances), torch.FloatTensor(test_thresholds), torch.FloatTensor(test_targets)), batch_size=batch_size, shuffle=True)
    
    return train_loader, test_loader, min_card, max_card
        

In [7]:
import torch
import torch.utils.data

train_loaders = []
test_loaders = []
min_cards = []
max_cards = []
for cluster_id in range(cluster_size):
    print (cluster_id)
    train, test, min_card, max_card = prepare_for_cluster(cluster_id)
    train_loaders.append(train)
    test_loaders.append(test)
    min_cards.append(min_card)
    max_cards.append(max_card)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
def normalize(labels, mini, maxi):
    return (torch.log(labels) - mini) / (maxi - mini)

def unnormalize(labels, mini, maxi):
    return torch.exp(labels * (maxi - mini) + mini)


## BaseLine Local Model

In [8]:
from __future__ import print_function
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

queries_dimension = 256
hidden_num = 128

class Test_Model(nn.Module):
    
    def __init__(self):
        super(Test_Model, self).__init__()
        self.threshold1 = nn.Linear(1, hidden_num)
        self.threshold2 = nn.Linear(hidden_num, 1)

        self.cnn_layer1 = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=3, stride=1, padding=2), 
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=3, stride=3))
        
        self.cnn_layer2 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=3, stride=1, padding=2), 
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=3, stride=3))
        
        self.out1 = nn.Linear(368, hidden_num)
        self.out2 = nn.Linear(hidden_num, 1)
        
    def forward(self, query, threshold):
        
        query = query.unsqueeze(2).permute(0,2,1)
#         print (query.shape)
        threshold = F.relu(self.threshold1(threshold))
        threshold = self.threshold2(threshold)
#         print (threshold.shape)
        query = self.cnn_layer1(query)
        query = self.cnn_layer2(query)
        query = query.view(query.shape[0], -1)
#         print (query.shape)
        query = self.out1(query)
        
        output = self.out2(F.relu(query+threshold))
        
        return output

def begin_test_model():
#     models = []
#     errors = []
#     for idx in range(100):
    idx = 0
    print ('idx: {}'.format(idx))
    train = train_loaders[idx]
    test = test_loaders[idx]
    mini = min_cards[idx]
    maxi = max_cards[idx]
    episode = 5
    queries_dimension = 256
    model = Test_Model()
    opt = optim.Adam(model.parameters(), lr=0.01)
    error = test_model(model, opt, train, test, episode)
    models.append(model)
    errors.append(error)

def test_model(model, opt, train, test, episode):
    print ('size: {}'.format(len(train)))
    test_errors = []
    for e in range(episode):
        model.train()
        for batch_idx, (queries, _, thresholds, targets) in enumerate(train):
    #         print (torch.cat((queries, thresholds), 1)[0])
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
    #         print (targets)
            opt.zero_grad()
            
            estimates = model(queries, thresholds)
            
            loss = l1_loss(estimates, targets)
            loss.backward()
            opt.step()
#             if batch_idx % 100 == 0:
#                 print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
#                 print(model.cnn_layer1[0].weight)
            for p in model.parameters():
                p.data.clamp_(-2, 2)
        model.eval()
        test_loss = 0.0
        mse_error = 0.0
        q_mean = 0.0
        q_max = 0.0
        for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
            
            estimates = model(queries, thresholds)
            
            loss = l1_loss(estimates, targets)
            mse, qer_mean, qer_max = print_loss(estimates, targets)
            test_loss += loss.item()
            mse_error += mse.item()
            q_mean += qer_mean
            if qer_max > q_max:
                q_max = qer_max
        test_loss /= len(test)
        mse_error /= len(test)
        q_mean /= len(test)
        test_errors.append(q_mean)
        print ('Testing: Iteration {0}, Loss {1}, MSE_error {2}, Q_error_mean {3}, Q_error_max {4}'.format(e, test_loss, mse_error, q_mean, q_max))
    return np.mean(test_errors[-3:])

In [ ]:
begin_test_model()

## Stacked Model Definition

In [9]:
from __future__ import print_function
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# queries_dimension = 200
# hidden_num = 128

class Threshold_Model(nn.Module):
    
    def __init__(self):
        super(Threshold_Model, self).__init__()
        self.fc1 = nn.Linear(1, hidden_num)
        self.fc2 = nn.Linear(hidden_num, 1)
    
    def forward(self, threshold):
        t1 = F.relu(self.fc1(threshold))
        t2 = self.fc2(t1)
        return t2

class CNN_Model(nn.Module):
    
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, pool_type, pool_size):
        super(CNN_Model, self).__init__()
        if pool_type == 0:
            pool_layer = nn.MaxPool1d(kernel_size=pool_size, stride=pool_size)
        elif pool_type == 1:
            pool_layer = nn.AvgPool1d(kernel_size=pool_size, stride=pool_size)
        else:
            print ('CNN_Model Init Error, invalid pool_type {}'.format(pool_type))
            return
        self.layer = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding), 
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            pool_layer)
        
    def forward(self, inputs):
        hid = self.layer(inputs)
#         print (hid.shape)
#         hid = F.relu(self.n3(hid))
#         hid = F.relu(self.n4(hid))
#         hid = self.norm2(hid)
#         print (hid.shape)
#         out2 = self.fc(hid.view(out1.shape[0], -1))
        return hid

class Output_Model(nn.Module):
    
    def __init__(self, inputs_dim):
        super(Output_Model, self).__init__()
        self.fc1 = nn.Linear(inputs_dim, hidden_num)
        self.fc2 = nn.Linear(hidden_num, 1)
        
    def forward(self, queries, threshold):
        out1 = F.relu(self.fc1(queries))
        out2 = out1 + threshold
#         print ('out2: {0}, threshold: {1}'.format(out2.shape, threshold.shape))
        out3 = self.fc2(out2)
        return out3

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.nn1 = nn.Linear(queries_dimension+1, hidden_num)
        self.n1 = nn.Linear(hidden_num, hidden_num)
        self.n2 = nn.Linear(hidden_num, hidden_num)
#         self.n3 = nn.Linear(hidden_num, hidden_num)
#         self.n4 = nn.Linear(hidden_num, hidden_num)
        self.nn2 = nn.Linear(hidden_num, 1)
        
    def forward(self, queries, threshold):
        out1 = F.relu(self.nn1(torch.cat([queries, threshold],1)))
        hid = out1
        hid = F.relu(self.n1(hid))
        hid = F.relu(self.n2(hid))
#         hid = F.relu(self.n3(hid))
#         hid = F.relu(self.n4(hid))
#         hid = self.norm2(hid)
        out2 = self.nn2(hid)
        return out2

def loss_fn(estimates, targets, mini, maxi):
    est = unnormalize(estimates, mini, maxi)
    print (torch.cat((est, targets), 1))
    return F.mse_loss(est, targets)

def l1_loss(estimates, targets, eps=1e-5):
    return F.smooth_l1_loss(estimates, torch.log(targets))

def mse_loss(estimates, targets, eps=1e-5):
#     print (torch.cat((estimates, targets), 1))
    return F.mse_loss(estimates, torch.log(targets))

def qerror_loss(preds, targets, mini, maxi):
    qerror = []
    preds = unnormal1ize_label(preds, mini, maxi)
#     print (torch.cat((preds, targets), 1))
    for i in range(len(targets)):
        if (preds[i] > targets[i]).cpu().data.numpy()[0]:
            qerror.append(preds[i]/targets[i])
        else:
            qerror.append(targets[i]/(preds[i] + 0.1))
    return torch.mean(torch.cat(qerror) ** 2)

def print_loss(estimates, targets):
    esti = torch.exp(estimates)
#     print (torch.cat((estimates, esti, targets), 1))
    qerror = []
    for i in range(esti.shape[0]):
        if esti[i] > targets[i] + 0.1:
            qerror.append((esti[i] / (targets[i] + 0.1)).item())
        else:
            qerror.append(((targets[i] + 0.1) / esti[i]).item())
    
    return F.mse_loss(esti, targets), np.mean(qerror), np.max(qerror)

## Hyper-parameters Selection Methods

In [10]:
from random import sample

def construct_model():
    errors = []
    next_cnn_parameterss = []
    next_cnn_modelss = []
    next_output_models = []
    threshold_models = []
    for clus in range(cluster_size):
        print ('Begin Cluster: {}'.format(clus))
        idx = clus
        train = train_loaders[idx]
        test = test_loaders[idx]
        mini = min_cards[idx]
        maxi = max_cards[idx]
        prev_best_error = 100000.0
        cnn_parameters, cnn_models = [], []
        episode = 5
        queries_dimension = 256
        threshold_model = Threshold_Model()
        error, next_cnn_parameters, next_cnn_models,next_output_model = select_best_layer(prev_best_error, cnn_parameters, cnn_models, threshold_model, train, test, episode, queries_dimension)
        saved_error, saved_next_cnn_parameters, saved_next_cnn_models,saved_next_output_model = error, next_cnn_parameters, next_cnn_models,next_output_model
        while error is not None:
            saved_error, saved_next_cnn_parameters, saved_next_cnn_models,saved_next_output_model = error, next_cnn_parameters, next_cnn_models,next_output_model
            print ('Cluster: {}, Error: {}, CNN Layer Num: {}, Added CNN Layer: {}'.format(clus, error, len(next_cnn_parameters), next_cnn_parameters[-1]))
            error, next_cnn_parameters, next_cnn_models,next_output_model = select_best_layer(error, next_cnn_parameters, next_cnn_models, threshold_model, train, test, episode, queries_dimension)
        errors.append(saved_error)
        next_cnn_parameterss.append(saved_next_cnn_parameters)
        next_cnn_modelss.append(saved_next_cnn_models)
        next_output_models.append(saved_next_output_model)
        threshold_models.append(threshold_model)
    return errors, next_cnn_parameterss, next_cnn_modelss, next_output_models, threshold_models

class TunableParameters():
    
    def __init__(self, out_channel, kernel_size, stride, padding, pool_size, pool_type):
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.pool_type = pool_type
    
    def __repr__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)
 
    def __str__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)

def select_best_layer(prev_best_error, cnn_parameters, cnn_models, threshold_model, train, test, episode, queries_dimension):
    print ('Input Model Size: {}'.format(len(cnn_parameters)))
    if len(cnn_parameters) > 0:
        in_channel = cnn_parameters[-1].out_channel
    else:
        in_channel = 1
    in_size = queries_dimension
    for para in cnn_parameters:
        in_size = int((int((in_size - para.kernel_size + 2*(para.padding)) / para.stride) + 1) / para.pool_size)
        print(para.kernel_size, para.padding, para.stride, para.pool_size, in_size)
    
    if in_size < 10 or len(cnn_parameters) > 5:
        return None, None, None, None
    
    current_paras = []
    current_paras.append(TunableParameters(8,10,1,2,9,0))
    current_paras.append(TunableParameters(4,5,3,2,4,0))
    current_paras.append(TunableParameters(2,2,1,0,2,0))
    
#     for out_channel in [2,4,8]:
#         for kernel_size in [2,4,8]:
#             for stride in range(1, min(4, kernel_size)):
#                 for padding in [0,2]:
#                     for pool_size in [kernel_size,]:
#                         for pool_type in [0,]:
#                             current_paras.append(TunableParameters(out_channel,kernel_size,stride,padding,pool_size,pool_type))
    print ('Group of parameters: {}'.format(len(current_paras)))
    next_cnn_models = []
    next_cnn_parameters = []
    next_output_model = None
#     current_paras = sample(current_paras, 2)
    for para in current_paras:
        print (para)
        in_size_local = int((int((in_size - para.kernel_size + 2*(para.padding)) / para.stride) + 1) / para.pool_size)
        print (in_size_local, para.out_channel)
        if in_size_local < 10:
            continue
        output_model = Output_Model(in_size_local * para.out_channel)
        added_cnn_layer = CNN_Model(in_channel, para.out_channel, para.kernel_size, para.stride, para.padding, para.pool_type, para.pool_size)
        paras = [{"params": model.parameters()} for model in cnn_models]
        paras.append({"params": added_cnn_layer.parameters()})
        paras.append({"params": threshold_model.parameters()})
        paras.append({"params": output_model.parameters()})
        opt = optim.Adam(paras, lr=0.001)
        new_cnn_models = []
        for model in cnn_models:
            new_cnn_models.append(model)
        new_cnn_models.append(added_cnn_layer)
        error = train_and_test(new_cnn_models, threshold_model, output_model, opt, train, test, episode)
        if error < prev_best_error - 0.1:
            prev_best_error = error
            new_cnn_parameters = []
            for para_old in cnn_parameters:
                new_cnn_parameters.append(para_old)
            next_output_model = output_model
            new_cnn_parameters.append(para)
            print ('Update layer: {}'.format(len(new_cnn_parameters)))
            next_cnn_parameters = new_cnn_parameters
            next_cnn_models = new_cnn_models
    if len(next_cnn_models) == 0:
        return None, None, None, None
    return prev_best_error, next_cnn_parameters, next_cnn_models, next_output_model
        

def only_test(cnn_models, threshold_model, output_model, test):
    for model in cnn_models:
        model.eval()
    threshold_model.eval()
    output_model.eval()
    q_errors = []
    for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
        queries = Variable(queries)
        thresholds = Variable(thresholds)
        targets = Variable(targets)

        queries = queries.unsqueeze(2).permute(0,2,1)
        for model in cnn_models:
            queries = model(queries)
        threshold = threshold_model(thresholds)
        queries = queries.view(queries.shape[0], -1)
        estimates = output_model(queries, threshold)

        loss = l1_loss(estimates, targets)
        
        esti = torch.exp(estimates)
        for i in range(esti.shape[0]):
            if esti[i] > targets[i] + 0.1:
                q_errors.append((esti[i] / (targets[i] + 0.1)).item())
            else:
                q_errors.append(((targets[i] + 0.1) / esti[i]).item())
    mean = np.mean(q_errors)
    percent90 = np.percentile(q_errors, 90)
    percent95 = np.percentile(q_errors, 95)
    percent99 = np.percentile(q_errors, 99)
    median = np.median(q_errors)
    maxi = np.max(q_errors)
    print ('Testing: Mean Error {}, Median Error {}, 90 Percent {}, 95 Percent {}, 99 Percent {}, Max Percent {}'
           .format(mean, median, percent90, percent95, percent99, maxi))
    
    
def train_and_test(cnn_models, threshold_model, output_model, opt, train, test, episode):
    print ('size: {}'.format(len(train)))
    test_errors = []
    for e in range(episode):
        for model in cnn_models:
            model.train()
        threshold_model.train()
        output_model.train()
        for batch_idx, (queries, _, thresholds, targets) in enumerate(train):
    #         print (torch.cat((queries, thresholds), 1)[0])
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
    #         print (targets)
            opt.zero_grad()
            queries = queries.unsqueeze(2).permute(0,2,1)
            for model in cnn_models:
                queries = model(queries)
            threshold = threshold_model(thresholds)
            queries = queries.view(queries.shape[0], -1)
            estimates = output_model(queries, threshold)
            
            loss = l1_loss(estimates, targets)
            loss.backward()
            opt.step()
#             if batch_idx % 100 == 0:
#                 print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
#                 print(cnn_models[0].layer[0].weight.grad)
            for p in model.parameters():
                p.data.clamp_(-2, 2)
            next(threshold_model.fc1.parameters()).data.clamp_(0)
            next(threshold_model.fc2.parameters()).data.clamp_(0)
            next(output_model.fc2.parameters()).data.clamp_(0)
        for model in cnn_models:
            model.eval()
        threshold_model.eval()
        output_model.eval()
        test_loss = 0.0
        mse_error = 0.0
        q_mean = 0.0
        q_max = 0.0
        for batch_idx, (queries, _, thresholds, targets) in enumerate(test):
            queries = Variable(queries)
            thresholds = Variable(thresholds)
            targets = Variable(targets)
            
            queries = queries.unsqueeze(2).permute(0,2,1)
            for model in cnn_models:
                queries = model(queries)
            threshold = threshold_model(thresholds)
            queries = queries.view(queries.shape[0], -1)
            estimates = output_model(queries, threshold)
            
            loss = l1_loss(estimates, targets)
            mse, qer_mean, qer_max = print_loss(estimates, targets)
            test_loss += loss.item()
            mse_error += mse.item()
            q_mean += qer_mean
            if qer_max > q_max:
                q_max = qer_max
        test_loss /= len(test)
        mse_error /= len(test)
        q_mean /= len(test)
        test_errors.append(q_mean)
        print ('Testing: Iteration {0}, Loss {1}, MSE_error {2}, Q_error_mean {3}, Q_error_max {4}'.format(e, test_loss, mse_error, q_mean, q_max))
    return np.mean(test_errors[-3:])

## Select Model Hyper-parameters

In [11]:
errors, next_cnn_parameterss, next_cnn_modelss, next_output_models, threshold_models = construct_model()


Begin Cluster: 0
Input Model Size: 0
Group of parameters: 3
8 10 1 2 9 0
27 8
size: 282
Testing: Iteration 0, Loss 0.02378397497877076, MSE_error 611618.4155405406, Q_error_mean 1.2031277511268854, Q_error_max 2.066420793533325
Testing: Iteration 1, Loss 0.007336945386251082, MSE_error 186925.40255489864, Q_error_mean 1.104544916303476, Q_error_max 1.5804516077041626
Testing: Iteration 2, Loss 0.0024933574620533633, MSE_error 58867.56213048987, Q_error_mean 1.0585669081884663, Q_error_max 1.300291657447815
Testing: Iteration 3, Loss 0.0014975644597733343, MSE_error 34247.79719172297, Q_error_mean 1.0447677718223751, Q_error_max 1.2425119876861572
Testing: Iteration 4, Loss 0.0012267638970090932, MSE_error 27815.228885135137, Q_error_mean 1.0401871259626303, Q_error_max 1.2200522422790527
Update layer: 1
4 5 3 2 4 0
21 4
size: 282
Testing: Iteration 0, Loss 0.0037824146267077005, MSE_error 93568.29391891892, Q_error_mean 1.0736628037474647, Q_error_max 1.4016895294189453
Testing: Iterat

Testing: Iteration 2, Loss 0.0003671788325772394, MSE_error 6674.049026076858, Q_error_mean 1.0215330779602803, Q_error_max 1.1299445629119873
Testing: Iteration 3, Loss 0.00035092989344628076, MSE_error 6313.150176177153, Q_error_mean 1.020480643404094, Q_error_max 1.1322253942489624
Testing: Iteration 4, Loss 0.0003505013104389749, MSE_error 6264.042454075169, Q_error_mean 1.020209978372362, Q_error_max 1.130624771118164
Cluster: 2, Error: 1.041186949267547, CNN Layer Num: 1, Added CNN Layer: 8 10 1 2 9 0
Input Model Size: 1
10 2 1 9 27
Group of parameters: 3
8 10 1 2 9 0
2 8
4 5 3 2 4 0
2 4
2 2 1 0 2 0
13 2
size: 282
Testing: Iteration 0, Loss 0.0011761417530628073, MSE_error 22375.908018369933, Q_error_mean 1.03936303169089, Q_error_max 1.2161006927490234
Testing: Iteration 1, Loss 0.0007294754441072409, MSE_error 13515.429317989865, Q_error_mean 1.0309794656177227, Q_error_max 1.1496107578277588
Testing: Iteration 2, Loss 0.0005473879518339763, MSE_error 10272.346890836148, Q_erro

Testing: Iteration 3, Loss 0.0010971704158126503, MSE_error 29410.619747677363, Q_error_mean 1.038240936534429, Q_error_max 1.21102774143219
Testing: Iteration 4, Loss 0.0008836769368310736, MSE_error 24921.057274070947, Q_error_mean 1.033911995325439, Q_error_max 1.188759684562683
Update layer: 1
4 5 3 2 4 0
21 4
size: 282
Testing: Iteration 0, Loss 0.004349290685904388, MSE_error 116598.57622466216, Q_error_mean 1.0788082614339687, Q_error_max 1.3800971508026123
Testing: Iteration 1, Loss 0.0018647602079687891, MSE_error 49464.23580025338, Q_error_mean 1.0501239568473317, Q_error_max 1.2565847635269165
Testing: Iteration 2, Loss 0.001132902261373159, MSE_error 30873.83688766892, Q_error_mean 1.038951709929451, Q_error_max 1.2033177614212036
Testing: Iteration 3, Loss 0.0009124029177631175, MSE_error 24293.127824113177, Q_error_mean 1.0344774045327023, Q_error_max 1.1916420459747314
Testing: Iteration 4, Loss 0.0007523779420660356, MSE_error 20094.959828969593, Q_error_mean 1.03116583

Testing: Iteration 0, Loss 0.0008317227380954333, MSE_error 20889.24340160473, Q_error_mean 1.034510957177829, Q_error_max 1.1885392665863037
Testing: Iteration 1, Loss 0.0006034493570631319, MSE_error 14587.07078758446, Q_error_mean 1.0277771843169388, Q_error_max 1.170896291732788
Testing: Iteration 2, Loss 0.0005486968808769676, MSE_error 13114.342417124155, Q_error_mean 1.0260203139151673, Q_error_max 1.1546950340270996
Testing: Iteration 3, Loss 0.0005078847579834228, MSE_error 12089.849055109797, Q_error_mean 1.0249024584285311, Q_error_max 1.1443431377410889
Testing: Iteration 4, Loss 0.0004087611989627869, MSE_error 10083.28514305321, Q_error_mean 1.0230947545905773, Q_error_max 1.1208970546722412
Begin Cluster: 8
Input Model Size: 0
Group of parameters: 3
8 10 1 2 9 0
27 8
size: 282
Testing: Iteration 0, Loss 0.007870527273799116, MSE_error 19952.354808910473, Q_error_mean 1.1103868213280834, Q_error_max 1.4759960174560547
Testing: Iteration 1, Loss 0.002515761002369628, MSE_e

Testing: Iteration 2, Loss 0.0008861762762535363, MSE_error 2426.2584047059754, Q_error_mean 1.034151192867776, Q_error_max 1.2060314416885376
Testing: Iteration 3, Loss 0.0007199763518607093, MSE_error 1905.1045004354942, Q_error_mean 1.030250999094868, Q_error_max 1.196092128753662
Testing: Iteration 4, Loss 0.0006182691321481724, MSE_error 1704.4253902950802, Q_error_mean 1.0286634053304993, Q_error_max 1.1526408195495605
2 2 1 0 2 0
127 2
size: 282
Testing: Iteration 0, Loss 0.001399864739977528, MSE_error 3731.347240551098, Q_error_mean 1.043140578914333, Q_error_max 1.2215354442596436
Testing: Iteration 1, Loss 0.0008960166329366935, MSE_error 2499.545121476457, Q_error_mean 1.03531721589231, Q_error_max 1.163670301437378
Testing: Iteration 2, Loss 0.0006589128622451697, MSE_error 1732.41082598712, Q_error_mean 1.0286739158091713, Q_error_max 1.16855788230896
Testing: Iteration 3, Loss 0.0005663240259997798, MSE_error 1505.5221821552998, Q_error_mean 1.026775539487701, Q_error_ma

...

Testing: Iteration 0, Loss 0.0007522028117327372, MSE_error 6276.1454550253375, Q_error_mean 1.0315731385143827, Q_error_max 1.1654870510101318
Testing: Iteration 1, Loss 0.0005213477994107667, MSE_error 4401.7377896695525, Q_error_mean 1.026102312714667, Q_error_max 1.1368632316589355
Testing: Iteration 2, Loss 0.00043941741908359264, MSE_error 3770.0896754909204, Q_error_mean 1.0241040432498463, Q_error_max 1.118422031402588
Testing: Iteration 3, Loss 0.0005760243511758745, MSE_error 4679.0965015308275, Q_error_mean 1.0273007433609784, Q_error_max 1.1441500186920166
Testing: Iteration 4, Loss 0.00034621123081337457, MSE_error 2931.9130628431167, Q_error_mean 1.0211147179550215, Q_error_max 1.117497205734253
Cluster: 87, Error: 1.0398372802673628, CNN Layer Num: 1, Added CNN Layer: 8 10 1 2 9 0
Input Model Size: 1
10 2 1 9 27
Group of parameters: 3
8 10 1 2 9 0
2 8
4 5 3 2 4 0
2 4
2 2 1 0 2 0
13 2
size: 282
Testing: Iteration 0, Loss 0.0017049259779899305, MSE_error 15006.417797191723

Testing: Iteration 1, Loss 0.005026972396425097, MSE_error 226866.44763513515, Q_error_mean 1.0848781736819326, Q_error_max 1.4854273796081543
Testing: Iteration 2, Loss 0.0024037516763987573, MSE_error 112224.55173141892, Q_error_mean 1.0578918585528594, Q_error_max 1.3162403106689453
Testing: Iteration 3, Loss 0.0015546312772769582, MSE_error 68693.22439822635, Q_error_mean 1.0456015784922685, Q_error_max 1.2699779272079468
Testing: Iteration 4, Loss 0.0013354119366487942, MSE_error 61027.55642947635, Q_error_mean 1.0425503742755264, Q_error_max 1.2351741790771484
Update layer: 1
4 5 3 2 4 0
21 4
size: 282
Testing: Iteration 0, Loss 0.0034004801261314265, MSE_error 150783.3255912162, Q_error_mean 1.069613154436386, Q_error_max 1.312707781791687
Testing: Iteration 1, Loss 0.0016000190375347596, MSE_error 70875.78937922297, Q_error_mean 1.0466580055528194, Q_error_max 1.2632653713226318
Testing: Iteration 2, Loss 0.0012388871046651558, MSE_error 55099.45951224662, Q_error_mean 1.040694

Testing: Iteration 3, Loss 0.0007883006780773301, MSE_error 24005.566485430743, Q_error_mean 1.0321058157982457, Q_error_max 1.1719123125076294
Testing: Iteration 4, Loss 0.0007151697881872187, MSE_error 22057.233319256757, Q_error_mean 1.0307006032608852, Q_error_max 1.1665781736373901
Cluster: 92, Error: 1.0453795923406746, CNN Layer Num: 1, Added CNN Layer: 8 10 1 2 9 0
Input Model Size: 1
10 2 1 9 27
Group of parameters: 3
8 10 1 2 9 0
2 8
4 5 3 2 4 0
2 4
2 2 1 0 2 0
13 2
size: 282
Testing: Iteration 0, Loss 0.002268459438660962, MSE_error 70455.50876266892, Q_error_mean 1.0555648069098793, Q_error_max 1.4375907182693481
Testing: Iteration 1, Loss 0.0012338706748434216, MSE_error 39138.44172297297, Q_error_mean 1.039729116950184, Q_error_max 1.2795381546020508
Testing: Iteration 2, Loss 0.0009295937795556981, MSE_error 27900.51905616554, Q_error_mean 1.0340998998239033, Q_error_max 1.2139030694961548
Testing: Iteration 3, Loss 0.0007301862493182558, MSE_error 22159.266443201013, Q_

Testing: Iteration 4, Loss 0.000955663832231752, MSE_error 49439.27227618243, Q_error_mean 1.035341754291408, Q_error_max 1.1825296878814697
Update layer: 1
4 5 3 2 4 0
21 4
size: 282
Testing: Iteration 0, Loss 0.0043380190555409, MSE_error 243722.48099662163, Q_error_mean 1.0791753953121401, Q_error_max 1.425943374633789
Testing: Iteration 1, Loss 0.0019053617064413186, MSE_error 100260.95893158784, Q_error_mean 1.051252298648595, Q_error_max 1.2588527202606201
Testing: Iteration 2, Loss 0.0014608205481734429, MSE_error 78878.07041807432, Q_error_mean 1.0448280380334, Q_error_max 1.2219175100326538
Testing: Iteration 3, Loss 0.0011543692986329865, MSE_error 59702.64590371621, Q_error_mean 1.039208436611335, Q_error_max 1.2232444286346436
Testing: Iteration 4, Loss 0.0009857736373748128, MSE_error 51692.11650126689, Q_error_mean 1.0361897930301525, Q_error_max 1.2054654359817505
2 2 1 0 2 0
127 2
size: 282
Testing: Iteration 0, Loss 0.0018124665917098723, MSE_error 98702.97080869932, Q

Testing: Iteration 0, Loss 0.001039866406810344, MSE_error 31502.470755912163, Q_error_mean 1.0370275142879501, Q_error_max 1.1895591020584106
Testing: Iteration 1, Loss 0.0006053402168028107, MSE_error 18219.912241342907, Q_error_mean 1.0277913906108085, Q_error_max 1.148205280303955
Testing: Iteration 2, Loss 0.0004783463978988899, MSE_error 14973.843195734797, Q_error_mean 1.025277231228412, Q_error_max 1.11808180809021
Testing: Iteration 3, Loss 0.00034536562334569926, MSE_error 10487.966631915118, Q_error_mean 1.0210249408500622, Q_error_max 1.1166198253631592
Testing: Iteration 4, Loss 0.00035903109803896497, MSE_error 10597.63024572424, Q_error_mean 1.0208430749590736, Q_error_max 1.120665192604065
Begin Cluster: 98
Input Model Size: 0
Group of parameters: 3
8 10 1 2 9 0
27 8
size: 282
Testing: Iteration 0, Loss 0.011595571023487562, MSE_error 152430.9522804054, Q_error_mean 1.1331339234083488, Q_error_max 1.8362990617752075
Testing: Iteration 1, Loss 0.003935992893939083, MSE_e

## Final Training

In [12]:
for idx in range(cluster_size):
    print (idx)
    test = test_loaders[idx]
    train = train_loaders[idx]
    paras = [{"params": model.parameters()} for model in next_cnn_modelss[idx]]
    paras.append({"params": threshold_models[idx].parameters()})
    paras.append({"params": next_output_models[idx].parameters()})
    opt = optim.Adam(paras, lr=0.001)
    episode = 2
    train_and_test(next_cnn_modelss[idx], threshold_models[idx], next_output_models[idx], opt, train, test, episode)
    only_test(next_cnn_modelss[idx], threshold_models[idx], next_output_models[idx], test)

0
size: 282
Testing: Iteration 0, Loss 0.0004265154554584139, MSE_error 9741.291015625, Q_error_mean 1.023427529852032, Q_error_max 1.1477950811386108
Testing: Iteration 1, Loss 0.0004119099046235803, MSE_error 9461.51537426098, Q_error_mean 1.0232883700849236, Q_error_max 1.1435797214508057
Testing: Mean Error 1.0233009532094002, Median Error 1.0217093229293823, 90 Percent 1.0404659509658813, 95 Percent 1.0579612255096436, 99 Percent 1.0877765417099, Max Percent 1.1435797214508057
1
size: 282
Testing: Iteration 0, Loss 0.0006699830913488325, MSE_error 19302.42896167652, Q_error_mean 1.0296805046523947, Q_error_max 1.1874920129776
Testing: Iteration 1, Loss 0.0006613438977306155, MSE_error 19312.927312077703, Q_error_mean 1.0302529762549377, Q_error_max 1.1805182695388794
Testing: Mean Error 1.0302652060909538, Median Error 1.030691146850586, 90 Percent 1.047955870628357, 95 Percent 1.0702170133590698, 99 Percent 1.1083757877349854, Max Percent 1.1805182695388794
2
size: 282
Testing: I

Testing: Mean Error 1.0211855004712636, Median Error 1.0208816528320312, 90 Percent 1.0348716974258423, 95 Percent 1.0504950284957886, 99 Percent 1.0770612955093384, Max Percent 1.1265857219696045
17
size: 282
Testing: Iteration 0, Loss 0.00026900007572761307, MSE_error 1078.0878452610325, Q_error_mean 1.0189803989755142, Q_error_max 1.1127961874008179
Testing: Iteration 1, Loss 0.0002699110985771325, MSE_error 1090.8984358504013, Q_error_mean 1.0194602137985262, Q_error_max 1.1082712411880493
Testing: Mean Error 1.019454394574879, Median Error 1.0189878940582275, 90 Percent 1.0302670001983643, 95 Percent 1.0417606830596924, 99 Percent 1.065043330192566, Max Percent 1.1082712411880493
18
size: 282
Testing: Iteration 0, Loss 0.0004565474255532781, MSE_error 1944.1632558461781, Q_error_mean 1.0245475878032881, Q_error_max 1.151144027709961
Testing: Iteration 1, Loss 0.00043933378254076014, MSE_error 1855.4379222972973, Q_error_mean 1.0235473512344666, Q_error_max 1.151432752609253
Testin

Testing: Iteration 1, Loss 0.0005395219140572826, MSE_error 16448.409496410473, Q_error_mean 1.0262048082375848, Q_error_max 1.1695469617843628
Testing: Mean Error 1.026205321596492, Median Error 1.021382451057434, 90 Percent 1.04988694190979, 95 Percent 1.070075511932373, 99 Percent 1.1045910120010376, Max Percent 1.1695469617843628
34
size: 282
Testing: Iteration 0, Loss 0.0003807675408048404, MSE_error 4501.112311285895, Q_error_mean 1.0226633034117922, Q_error_max 1.1352885961532593
Testing: Iteration 1, Loss 0.0003796504663753424, MSE_error 4406.281725084459, Q_error_mean 1.0213817291943408, Q_error_max 1.1407711505889893
Testing: Mean Error 1.0213900153480826, Median Error 1.016244888305664, 90 Percent 1.0432963371276855, 95 Percent 1.0598713159561157, 99 Percent 1.0880831480026245, Max Percent 1.1407711505889893
35
size: 282
Testing: Iteration 0, Loss 0.0003433144626575774, MSE_error 3696.47162360114, Q_error_mean 1.0216476105002534, Q_error_max 1.1268465518951416
Testing: Itera

Testing: Iteration 0, Loss 0.0004507239830894144, MSE_error 7668.565660631334, Q_error_mean 1.0243921311622537, Q_error_max 1.1504377126693726
Testing: Iteration 1, Loss 0.0004420297553076887, MSE_error 7443.143204972551, Q_error_mean 1.0234788666554802, Q_error_max 1.1521795988082886
Testing: Mean Error 1.0234826148747385, Median Error 1.0186729431152344, 90 Percent 1.0454181432724, 95 Percent 1.0635132789611816, 99 Percent 1.0943670272827148, Max Percent 1.1521795988082886
51
size: 282
Testing: Iteration 0, Loss 0.00036873439023258616, MSE_error 2981.8117411845437, Q_error_mean 1.0211221093588785, Q_error_max 1.1385055780410767
Testing: Iteration 1, Loss 0.0003451170970849391, MSE_error 2823.1712184596704, Q_error_mean 1.0212106754172694, Q_error_max 1.1312994956970215
Testing: Mean Error 1.021219239842628, Median Error 1.0192762613296509, 90 Percent 1.037561297416687, 95 Percent 1.0535200834274292, 99 Percent 1.0806646347045898, Max Percent 1.1312994956970215
52
size: 282
Testing: I

Testing: Mean Error 1.0204480834254601, Median Error 1.0166966915130615, 90 Percent 1.0389376878738403, 95 Percent 1.054569125175476, 99 Percent 1.0811467170715332, Max Percent 1.1306853294372559
67
size: 282
Testing: Iteration 0, Loss 0.00033434046250673617, MSE_error 1778.6142561629013, Q_error_mean 1.0207618089468293, Q_error_max 1.1296381950378418
Testing: Iteration 1, Loss 0.0003234985547228339, MSE_error 1728.8318745381123, Q_error_mean 1.0206145779733118, Q_error_max 1.1259673833847046
Testing: Mean Error 1.0206287839532304, Median Error 1.0194181203842163, 90 Percent 1.0355507135391235, 95 Percent 1.0509612560272217, 99 Percent 1.0771571397781372, Max Percent 1.1259673833847046
68
size: 282
Testing: Iteration 0, Loss 0.000575034848864291, MSE_error 19350.22975612331, Q_error_mean 1.028889439559567, Q_error_max 1.1543748378753662
Testing: Iteration 1, Loss 0.0005271101950170674, MSE_error 17273.26021431588, Q_error_mean 1.0262819738756563, Q_error_max 1.1608442068099976
Testing:

Testing: Iteration 1, Loss 0.0003531803430067501, MSE_error 2670.852492873733, Q_error_mean 1.0210845366095167, Q_error_max 1.1345634460449219
Testing: Mean Error 1.0210913882723875, Median Error 1.0172909498214722, 90 Percent 1.0399669408798218, 95 Percent 1.0560675859451294, 99 Percent 1.083457589149475, Max Percent 1.1345634460449219
84
size: 282
Testing: Iteration 0, Loss 0.0005368839735416947, MSE_error 7413.85094884924, Q_error_mean 1.0265695611266672, Q_error_max 1.16619074344635
Testing: Iteration 1, Loss 0.0005251094187622437, MSE_error 7270.778834987331, Q_error_mean 1.0263790897521619, Q_error_max 1.1632649898529053
Testing: Mean Error 1.0263880399518273, Median Error 1.0247308015823364, 90 Percent 1.0455880165100098, 95 Percent 1.065453052520752, 99 Percent 1.099404215812683, Max Percent 1.1632649898529053
85
size: 282
Testing: Iteration 0, Loss 0.00042833171500132193, MSE_error 5144.01453626478, Q_error_mean 1.0231528784072883, Q_error_max 1.1494038105010986
Testing: Itera

## Save The Model

In [13]:
for idx in range(cluster_size):
    states = {}
    for idd, cnn_model in enumerate(next_cnn_modelss[idx]):
        states['cnn_model_state_dict_' + str(idd)] = cnn_model.state_dict()
    states['threshold_model_state_dict'] = threshold_models[idx].state_dict()
    states['output_model_state_dict'] = next_output_models[idx].state_dict()
    torch.save(states, '/home/sunji/ANN/nytimes_256_angular/saved_models/local_nytimes_256_angular_cluster_' + str(idx) + '.model')
    
    

In [14]:
import pickle
with open('/home/sunji/ANN/nytimes_256_angular/saved_models/cnn_hyper_parameters.hyperpara', 'w') as handle:
    for idx in range(cluster_size):
        handle.write(';'.join(str(x) for x in next_cnn_parameterss[idx]))
        handle.write('\n')

## Model Test

In [ ]:
model = Model()
model.load_state_dict(torch.load('MDN_model'))
model.eval()